# Este notebook faz parte da série de reportagens [Especulação imobiliária em Fortaleza](https://mais.opovo.com.br/reportagens-especiais/fortaleza-especulacao-imobiliaria)

Série de reportagens aborda as novas paisagens de Fortaleza, a partir da análise da ocupação dos bairros Mucuripe, Meireles, Cocó e Sabiaguaba. Os códigos e bases de dados exploradas nesta série estão descritas aqui. 

In [1]:

!pip install geopandas
!pip install googlemaps
import pandas as pd
import geopandas as gpd
import os
import googlemaps
from shapely import wkt


In [2]:
pd.options.display.float_format = '{:.2f}'.format

# Leitura dos dados

In [3]:
path = 'bases_de_dados'
path2 = ''

In [4]:
bairros = pd.read_csv(path+'Bairros_de_Fortaleza.csv')

In [5]:
bairros.columns

Index(['FID', 'id', 'The_geom', 'nome', 'cod_ibge', 'cod_bairro', 'area_ha',
       'ser_antiga', 'ser_atual', 'cod_regiao', 'territorio', 'fonte',
       'ano_ref', 'data', 'epsg_codif'],
      dtype='object')

In [6]:
bairros.head(1)

,FID,id,The_geom,nome,cod_ibge,cod_bairro,area_ha,ser_antiga,ser_atual,cod_regiao,territorio,fonte,ano_ref,data,epsg_codif
0,vw_Fortaleza_Bairros.fid--16360be6_1885356760e...,77,MULTIPOLYGON (((-38.554614978346514 -3.8094727...,Rachel de Queiroz,2304400077,77,196.88,SER IV,Secretaria Executiva Regional 8,VI,19,IPLANFOR,2019,2022-11-13,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...


In [7]:
bairros = bairros[['id','nome','cod_ibge','cod_bairro','area_ha','ser_atual','cod_regiao','The_geom']]

In [8]:
bairros['The_geom'] = bairros.The_geom.apply(wkt.loads)
gdf_bairros = gpd.GeoDataFrame(bairros, geometry='The_geom', crs='EPSG:4326')

<ipython-input-8-400ae900e51b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bairros['The_geom'] = bairros.The_geom.apply(wkt.loads)


## IDH

In [9]:
idh = pd.read_csv(path+'IDH-_Classificação.csv')

In [10]:
idh = idh[['cod_bairro','idh_educ','idh_longev','idh_renda','idh','idh_calssi']]

In [11]:
bairros_idh = pd.merge(bairros,idh, on='cod_bairro', how='outer', indicator =True)

In [12]:
bairros_idh._merge.value_counts()

both          116
left_only       5
right_only      3
Name: _merge, dtype: int64

In [13]:
bairros_idh.loc[bairros_idh._merge=='left_only']

,id,nome,cod_ibge,cod_bairro,area_ha,ser_atual,cod_regiao,The_geom,idh_educ,idh_longev,idh_renda,idh,idh_calssi,_merge
6,121.00,Boa Vista,2304400125.00,125,295.75,Secretaria Executiva Regional 8,VI,MULTIPOLYGON (((-38.53141684209245 -3.79757788...,NaN,NaN,NaN,NaN,NaN,left_only
68,120.00,Aracapé,2304400124.00,124,217.94,Secretaria Executiva Regional 10,X,MULTIPOLYGON (((-38.58713492324229 -3.81952490...,NaN,NaN,NaN,NaN,NaN,left_only
116,118.00,Olavo Oliveira,2304400122.00,122,71.17,Secretaria Executiva Regional 3,IV,MULTIPOLYGON (((-38.59616668182891 -3.72534242...,NaN,NaN,NaN,NaN,NaN,left_only
118,119.00,Santa Maria,2304400123.00,123,196.96,Secretaria Executiva Regional 9,IX,MULTIPOLYGON (((-38.49419030206032 -3.83901007...,NaN,NaN,NaN,NaN,NaN,left_only
120,117.00,Novo Mondubim,2304400121.00,121,175.45,Secretaria Executiva Regional 10,X,MULTIPOLYGON (((-38.59428171899377 -3.80758093...,NaN,NaN,NaN,NaN,NaN,left_only


In [14]:
bairros_idh.loc[bairros_idh._merge=='right_only']

,id,nome,cod_ibge,cod_bairro,area_ha,ser_atual,cod_regiao,The_geom,idh_educ,idh_longev,idh_renda,idh,idh_calssi,_merge
121,NaN,NaN,NaN,102,NaN,NaN,NaN,NaN,0.99,0.85,0.29,0.63,"0,600 - 0,699 / Médio",right_only
122,NaN,NaN,NaN,95,NaN,NaN,NaN,NaN,0.94,0.24,0.10,0.29,"0 - 0,499 / Muito Baixo",right_only
123,NaN,NaN,NaN,117,NaN,NaN,NaN,NaN,0.92,0.13,0.01,0.11,"0 - 0,499 / Muito Baixo",right_only


## Saúde, Educação e Assistência Social

In [15]:
merge_bairros = bairros.copy()
merge_bairros.tail(2)

,id,nome,cod_ibge,cod_bairro,area_ha,ser_atual,cod_regiao,The_geom
119,116,São Bento,2304400119,119,219.33,Secretaria Executiva Regional 6,VIII,MULTIPOLYGON (((-38.473760953231626 -3.8547543...
120,117,Novo Mondubim,2304400121,121,175.45,Secretaria Executiva Regional 10,X,MULTIPOLYGON (((-38.59428171899377 -3.80758093...


### Saúde

In [16]:
saude = pd.read_csv(path+'Equipamentos_de_Saúde.csv')

In [17]:
saude.head(2)

,FID,id,The_geom,fid,NOME,CNES,ENDEREÇO,CEP,NATUREZA,REGIONAL,BAIRRO,COORDENADO,EMAIL,TELEFONE,CONTATO,Fonte,ano_ref,epsg_codif
0,vw_EquipamentosSaude.fid--16360be6_1884ed137dc...,1,MULTIPOINT ((-38.539134700000005 -3.7528683999...,1,CAPS GERAL SER IV,2529041.00,Sem informação,Sem informação,CAPS,4,Jardim America,Sem informação,Sem informação,Sem informação,Sem informação,SMS,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
1,vw_EquipamentosSaude.fid--16360be6_1884ed137dc...,2,MULTIPOINT ((-38.488755 -3.83305599997293)),2,CAPS GERAL SER VI,2561263.00,Sem informação,Sem informação,CAPS,6,Messejana,Sem informação,Sem informação,Sem informação,Sem informação,SMS,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...


In [18]:
saude.CNES = saude.CNES.fillna(0)

In [19]:
saude.shape

(168, 18)

In [20]:
saude.NATUREZA.value_counts()

UAPS           116
HOSPITAL        21
CAPS            15
UPA             12
POLICLINICA      4
Name: NATUREZA, dtype: int64

In [21]:
saude['Descricao'] = saude.apply(lambda x: f'{x.NOME}, CNES: {str(int(x.CNES))}', axis=1)
saude['Tipo'] = saude.NATUREZA
saude = saude[['The_geom','Tipo','Descricao']]

In [22]:
samu = pd.read_csv(path+'Bases_do_SAMU.csv').rename(columns={'tipo':'Tipo','base_id':'Descricao', 'geom':'The_geom'})

In [23]:
samu.Tipo.value_counts()

Base do SAMU    15
Name: Tipo, dtype: int64

In [24]:
samu = samu[['The_geom','Tipo','Descricao']]

In [25]:
saude = pd.concat([saude,samu])
saude['Categoria'] = 'Equipamentos de Saúde'
saude

,The_geom,Tipo,Descricao,Categoria
0,MULTIPOINT ((-38.539134700000005 -3.7528683999...,CAPS,"CAPS GERAL SER IV, CNES: 2529041",Equipamentos de Saúde
1,MULTIPOINT ((-38.488755 -3.83305599997293)),CAPS,"CAPS GERAL SER VI, CNES: 2561263",Equipamentos de Saúde
2,MULTIPOINT ((-38.55091860000001 -3.74953919996...,CAPS,"CAPS GERAL SER III, CNES: 2651378",Equipamentos de Saúde
3,MULTIPOINT ((-38.602163100000006 -3.7940350999...,CAPS,"CAPS GERAL SER V, CNES: 3878562",Equipamentos de Saúde
4,MULTIPOINT ((-38.55451930000001 -3.74610619996...,CAPS,"CAPS AD SER III, CNES: 3950530",Equipamentos de Saúde
...,...,...,...,...
10,POINT (-38.57461962033301 -3.762324835937796),Base do SAMU,Hospital da Mulher,Equipamentos de Saúde
11,POINT (-38.53987044083168 -3.7538498533086457),Base do SAMU,Central de Abastecimento - CSA,Equipamentos de Saúde
12,POINT (-38.56896730442408 -3.8083529134780716),Base do SAMU,Maraponga,Equipamentos de Saúde
13,POINT (-38.513321926916674 -3.8104537968741328),Base do SAMU,Cajazeiras,Equipamentos de Saúde


In [26]:
saude['The_geom'] = saude.The_geom.apply(wkt.loads)
gdf_saude = gpd.GeoDataFrame(saude, geometry='The_geom', crs='EPSG:4326')

sjoin_bairros = gdf_saude.sjoin(gdf_bairros)
resumo_bairros = sjoin_bairros.groupby(['nome','Tipo']).size().reset_index()

resumo_bairros_total = sjoin_bairros.groupby('nome').size().reset_index()
resumo_bairros_total['Tipo'] = 'qtd_saude'


resumo_bairros = pd.concat([resumo_bairros_total, resumo_bairros])

resumo_bairros = resumo_bairros.pivot(index='nome',columns='Tipo').fillna(0).astype(int).reset_index()
resumo_bairros.columns = ['nome','Base do SAMU','CAPS','HOSPITAL','POLICLINICA','UAPS','UPA','qtd_saude']

merge_bairros = pd.merge(merge_bairros,resumo_bairros,how='outer')
merge_bairros = merge_bairros.fillna(0)
merge_bairros.qtd_saude = merge_bairros.qtd_saude.astype(int)
merge_bairros.head(2)

,id,nome,cod_ibge,cod_bairro,area_ha,ser_atual,cod_regiao,The_geom,Base do SAMU,CAPS,HOSPITAL,POLICLINICA,UAPS,UPA,qtd_saude
0,77,Rachel de Queiroz,2304400077,77,196.88,Secretaria Executiva Regional 8,VI,MULTIPOLYGON (((-38.554614978346514 -3.8094727...,0.00,1.00,0.00,0.00,2.00,1.00,4
1,23,José Bonifácio,2304400023,23,88.90,Secretaria Executiva Regional 4,V,MULTIPOLYGON (((-38.523928312458885 -3.7369035...,0.00,0.00,0.00,0.00,0.00,0.00,0


In [27]:
merge_bairros['qtd_saude'].sum()

183

In [28]:
df_pontos = sjoin_bairros.copy()

### Ass. Social

In [29]:
ass_social = pd.read_csv(path+'Equipamentos_de_Assistência_Social.csv').rename(columns = {'the_geom':'The_geom','nome':'Descricao','tipo':'Tipo'})

In [30]:
ass_social.describe()

,id,cod_bairro,ano_inauguracao,ano_ref
count,49.00,49.00,49.00,49.00
mean,25.00,55.55,2009.76,2020.00
std,14.29,38.31,5.11,0.00
min,1.00,4.00,2004.00,2020.00
25%,13.00,23.00,2005.00,2020.00
50%,25.00,54.00,2009.00,2020.00
75%,37.00,89.00,2014.00,2020.00
max,49.00,125.00,2020.00,2020.00


In [31]:
ass_social = ass_social[['The_geom','Tipo','Descricao']]

In [32]:
ass_social.Tipo.value_counts()

CRAS                                                                 27
CREAS                                                                 6
Unidade de Acolhimento Institucional para Crianças e Adolescentes     6
Centro de convivência e abrigo institucional                          2
Centro-Dia                                                            2
Centro POP                                                            2
Unidade de Acolhimento Pousada Social                                 2
Refeitório Social                                                     1
Unidade de Acolhimento                                                1
Name: Tipo, dtype: int64

In [33]:
lista_tipos = ['CRAS', 'CREAS', 'Centro de convivência e abrigo institucional',
       'Unidade de Acolhimento Institucional para Crianças e Adolescentes']

In [34]:
ass_social.loc[~ass_social.Tipo.isin(lista_tipos),'Tipo'] = 'Outros_Ass_Social'

In [35]:
ass_social.Tipo.value_counts()

CRAS                                                                 27
Outros_Ass_Social                                                     8
CREAS                                                                 6
Unidade de Acolhimento Institucional para Crianças e Adolescentes     6
Centro de convivência e abrigo institucional                          2
Name: Tipo, dtype: int64

In [36]:
caps = pd.read_csv(path+'CAPS.csv').rename(columns = {'geom':'The_geom','nome':'Descricao','tipo':'Tipo'})

In [37]:
caps.Tipo.value_counts()

Centro de Atenção Psicossocial    15
Name: Tipo, dtype: int64

In [38]:
caps.Tipo = 'CAPS'

In [39]:
caps = caps[['The_geom','Tipo','Descricao']]

ass_social = pd.concat([ass_social,caps])

ass_social['The_geom'] = ass_social.The_geom.apply(wkt.loads)

In [40]:
ass_social['Categoria'] = 'Equipamentos de Assistência Social'

In [41]:
gdf_ass_social = gpd.GeoDataFrame(ass_social, geometry='The_geom', crs='EPSG:4326')

sjoin_bairros = gdf_ass_social.sjoin(gdf_bairros)
resumo_bairros = sjoin_bairros.groupby(['nome','Tipo']).size().reset_index()

resumo_bairros_total = sjoin_bairros.groupby('nome').size().reset_index()
resumo_bairros_total['Tipo'] = 'qtd'


resumo_bairros = pd.concat([resumo_bairros_total, resumo_bairros])

resumo_bairros = resumo_bairros.pivot(index='nome',columns='Tipo').fillna(0).astype(int).reset_index()
resumo_bairros.columns = ['nome','CAPS','CRAS','CREAS','Centro de convivência e abrigo institucional',	'Outros_Ass_Social','Unidade de Acolhimento Institucional para Crianças e Adolescentes','qtd_ass_social']

resumo_bairros.loc[resumo_bairros.nome.str.contains('Jacarecanga'),'nome'] = 'Jacarecanga'

merge_bairros = pd.merge(resumo_bairros, merge_bairros,how='outer', on='nome')


merge_bairros = merge_bairros.fillna(0)
merge_bairros.qtd_ass_social = merge_bairros.qtd_ass_social.astype(int)
merge_bairros.head(2)

,nome,CAPS_x,CRAS,CREAS,Centro de convivência e abrigo institucional,Outros_Ass_Social,Unidade de Acolhimento Institucional para Crianças e Adolescentes,qtd_ass_social,id,cod_ibge,...,ser_atual,cod_regiao,The_geom,Base do SAMU,CAPS_y,HOSPITAL,POLICLINICA,UAPS,UPA,qtd_saude
0,Aldeota,1.00,0.00,0.00,0.00,0.00,0.00,1,2,2304400002,...,Secretaria Executiva Regional 2,III,MULTIPOLYGON (((-38.50884906778375 -3.74164933...,0.00,1.00,1.00,0.00,1.00,0.00,3
1,Amadeu Furtado,0.00,0.00,1.00,0.00,0.00,1.00,2,4,2304400004,...,Secretaria Executiva Regional 3,IV,MULTIPOLYGON (((-38.559446251425214 -3.7505106...,0.00,0.00,1.00,0.00,0.00,0.00,1


In [42]:
df_pontos = pd.concat([df_pontos,sjoin_bairros])
df_pontos.Categoria.value_counts()

Equipamentos de Saúde                 183
Equipamentos de Assistência Social     64
Name: Categoria, dtype: int64

### Educação

In [43]:
parque_estadual = pd.read_csv(path+'Parque_Escolar_Estadual.csv').rename(columns={'nome':'Descricao','geom':'The_geom'})
parque_estadual['Descricao'] = parque_estadual.apply(lambda x: f'({x["tipo"]}) {x["Descricao"]}',axis=1)
parque_estadual['Tipo'] = 'Parque Escolar Estadual'
parque_estadual = parque_estadual[['The_geom','Tipo','Descricao']]

In [44]:
parque_municipal = pd.read_csv(path+'Parque_Escolar_Municipal.csv')
parque_municipal['Descricao'] = parque_municipal.apply(lambda x: f'({x["tipo"]}) {x["nome"]}',axis=1)
parque_municipal['Tipo'] = 'Parque Escolar Municipal'
parque_municipal = parque_municipal[['The_geom','Tipo','Descricao']]

In [45]:
parque_escolar = pd.concat([parque_municipal,parque_estadual])
parque_escolar['Categoria'] = 'Parque Escolar Municipal e Estadual'

In [46]:
parque_escolar.Tipo.value_counts()

Parque Escolar Municipal    613
Parque Escolar Estadual     167
Name: Tipo, dtype: int64

In [47]:
parque_escolar['The_geom'] = parque_escolar.The_geom.apply(wkt.loads)

In [48]:
gdf_parque_escolar = gpd.GeoDataFrame(parque_escolar, geometry='The_geom', crs='EPSG:4326')

sjoin_bairros = gdf_parque_escolar.sjoin(gdf_bairros)
resumo_bairros = sjoin_bairros.groupby(['nome','Tipo']).size().reset_index()

resumo_bairros_total = sjoin_bairros.groupby('nome').size().reset_index()
resumo_bairros_total['Tipo'] = 'qtd'


resumo_bairros = pd.concat([resumo_bairros_total, resumo_bairros])

resumo_bairros = resumo_bairros.pivot(index='nome',columns='Tipo').fillna(0).astype(int).reset_index()
resumo_bairros.columns = ['nome','Parque Escolar Estadual',	'Parque Escolar Municipal','qtd_parque_escolar']

merge_bairros = pd.merge(merge_bairros,resumo_bairros,how='outer')
merge_bairros = merge_bairros.fillna(0)
merge_bairros.qtd_parque_escolar = merge_bairros.qtd_parque_escolar.astype(int)
merge_bairros.head(2)

,nome,CAPS_x,CRAS,CREAS,Centro de convivência e abrigo institucional,Outros_Ass_Social,Unidade de Acolhimento Institucional para Crianças e Adolescentes,qtd_ass_social,id,cod_ibge,...,Base do SAMU,CAPS_y,HOSPITAL,POLICLINICA,UAPS,UPA,qtd_saude,Parque Escolar Estadual,Parque Escolar Municipal,qtd_parque_escolar
0,Aldeota,1.00,0.00,0.00,0.00,0.00,0.00,1,2,2304400002,...,0.00,1.00,1.00,0.00,1.00,0.00,3,3.00,2.00,5
1,Amadeu Furtado,0.00,0.00,1.00,0.00,0.00,1.00,2,4,2304400004,...,0.00,0.00,1.00,0.00,0.00,0.00,1,2.00,1.00,3


In [49]:
df_pontos = pd.concat([df_pontos,sjoin_bairros])
df_pontos.Categoria.value_counts()

Parque Escolar Municipal e Estadual    779
Equipamentos de Saúde                  183
Equipamentos de Assistência Social      64
Name: Categoria, dtype: int64

In [50]:
df_pontos.Tipo.value_counts()

Parque Escolar Municipal                                             612
Parque Escolar Estadual                                              167
UAPS                                                                 116
CAPS                                                                  30
CRAS                                                                  27
HOSPITAL                                                              21
Base do SAMU                                                          15
UPA                                                                   12
Outros_Ass_Social                                                      8
CREAS                                                                  6
Unidade de Acolhimento Institucional para Crianças e Adolescentes      6
POLICLINICA                                                            4
Centro de convivência e abrigo institucional                           2
Name: Tipo, dtype: int64

### Praças e Equipamentos FUNDURB

In [51]:
pracas = pd.read_csv(path+'Praças_de_Fortaleza.csv').rename(columns = {'the_geom':'The_geom','nome':'nome_pca'})
pracas['Descricao'] = pracas.apply(lambda x: f'{x.nome_pop} ({x["area m2"]} m²). Nome Oficial: {x.nome_pca}',axis=1)
pracas['Tipo'] = 'Praças'
pracas = pracas[['The_geom','Tipo','Descricao']]

In [52]:
eqp_fundurb = pd.read_csv(path+'Equipamentos_-_FUNDURB.csv').rename(columns = {'Nome':'Descricao'})
eqp_fundurb['Tipo'] = 'Eqp FUNDURB'
eqp_fundurb = eqp_fundurb[['The_geom','Tipo','Descricao']]

In [53]:
pracas_fundurb = pd.concat([pracas, eqp_fundurb])
pracas_fundurb['Categoria'] = 'Praças e equipamentos FUNDURB'

In [54]:
pracas_fundurb['The_geom'] = pracas_fundurb.The_geom.apply(wkt.loads)

In [55]:
gdf_pracas = gpd.GeoDataFrame(pracas_fundurb, geometry='The_geom', crs='EPSG:4326')

sjoin_bairros = gdf_pracas.sjoin(gdf_bairros)
resumo_bairros = sjoin_bairros.groupby(['nome','Tipo']).size().reset_index()

resumo_bairros_total = sjoin_bairros.groupby('nome').size().reset_index()
resumo_bairros_total['Tipo'] = 'qtd'

resumo_bairros = pd.concat([resumo_bairros_total, resumo_bairros])

resumo_bairros = resumo_bairros.pivot(index='nome',columns='Tipo').fillna(0).astype(int).reset_index()
resumo_bairros.columns = ['nome','Eqp FUNDURB',	'Praças','qtd_pracas_fundurb']

merge_bairros = pd.merge(merge_bairros,resumo_bairros,how='outer')
merge_bairros = merge_bairros.fillna(0)
merge_bairros.qtd_parque_escolar = merge_bairros.qtd_parque_escolar.astype(int)
merge_bairros.head(2)

,nome,CAPS_x,CRAS,CREAS,Centro de convivência e abrigo institucional,Outros_Ass_Social,Unidade de Acolhimento Institucional para Crianças e Adolescentes,qtd_ass_social,id,cod_ibge,...,POLICLINICA,UAPS,UPA,qtd_saude,Parque Escolar Estadual,Parque Escolar Municipal,qtd_parque_escolar,Eqp FUNDURB,Praças,qtd_pracas_fundurb
0,Aldeota,1.00,0.00,0.00,0.00,0.00,0.00,1,2,2304400002,...,0.00,1.00,0.00,3,3.00,2.00,5,0.00,10.00,10.00
1,Amadeu Furtado,0.00,0.00,1.00,0.00,0.00,1.00,2,4,2304400004,...,0.00,0.00,0.00,1,2.00,1.00,3,0.00,2.00,2.00


In [56]:
df_pontos = pd.concat([df_pontos,sjoin_bairros])
df_pontos.Categoria.value_counts()

Parque Escolar Municipal e Estadual    779
Praças e equipamentos FUNDURB          525
Equipamentos de Saúde                  183
Equipamentos de Assistência Social      64
Name: Categoria, dtype: int64

In [57]:

df_pontos.Tipo.value_counts()

Parque Escolar Municipal                                             612
Praças                                                               490
Parque Escolar Estadual                                              167
UAPS                                                                 116
Eqp FUNDURB                                                           35
CAPS                                                                  30
CRAS                                                                  27
HOSPITAL                                                              21
Base do SAMU                                                          15
UPA                                                                   12
Outros_Ass_Social                                                      8
CREAS                                                                  6
Unidade de Acolhimento Institucional para Crianças e Adolescentes      6
POLICLINICA                                        

In [58]:
df_pontos.groupby(['Categoria','Tipo']).size().reset_index().to_csv('resumo_categorias.csv')

### População

In [59]:
teste = pd.read_csv(path+'Densidade_Populacional_por_Bairros.csv')
teste.loc[teste.cod_bairro ==77]

,FID,id,The_geom,cod_bairro,bairro,regional_a,cod_regiao,regiao_adm,territorio,area_km2,populacao_,den_km2,fonte,ano_ref,epsg_codif
84,vw_BairrosDensidade.fid--16360be6_1885356760e_...,77,MULTIPOLYGON (((-38.554614978346514 -3.8094727...,77,Dendê,SER IV,VI,SER 8,19,1.97,6215,3156.42,PMF/SMS/COVIS/CEVEPI. *Projeção populacional c...,2022,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...


In [60]:
df_pop = pd.read_csv(path+'Densidade_Populacional_por_Bairros.csv')[['bairro','populacao_']]
df_pop.columns = ['nome','pop_2010']

In [61]:
df_pop.shape

(121, 2)

In [62]:
df_pop

,nome,pop_2010
0,São Gerardo,15990
1,Aldeota,46701
2,Alvaro Weyne,26117
3,Amadeu Furtado,12901
4,Moura Brasil,4150
...,...,...
116,De Lourdes,3716
117,Olavo Oliveira,13403
118,Santa Maria,14709
119,São Bento,13189


In [63]:
merge_bairros.shape

(121, 28)

In [64]:
merge_bairros.loc[merge_bairros.nome.str.contains('Rach'),'nome'] = 'Dendê'

In [65]:
df_final_bairros = pd.merge(merge_bairros,df_pop, on='nome')

In [66]:
df_final_bairros = df_final_bairros.drop_duplicates()

In [67]:
df_final_bairros[['nome','pop_2010']]

,nome,pop_2010
0,Aldeota,46701
1,Amadeu Furtado,12901
2,Antônio Bezerra,28493
3,Barra do Ceará,79842
4,Barroso,32905
...,...,...
116,Planalto Ayrton Senna,43488
117,De Lourdes,3716
118,Olavo Oliveira,13403
119,Santa Maria,14709


In [68]:
# df_final_bairros[['nome','qtd_saude','qtd_ass_social','qtd_parque_escolar','qtd_pracas_fundurb','pop_2010']]

In [69]:
df_final_bairros['total'] = df_final_bairros['qtd_saude'] +df_final_bairros['qtd_ass_social'] +df_final_bairros['qtd_parque_escolar'] +df_final_bairros['qtd_pracas_fundurb']
df_final_bairros['total'] = df_final_bairros['total'].astype(int)

In [70]:
df_final_bairros[['nome','pop_2010','total']]

,nome,pop_2010,total
0,Aldeota,46701,19
1,Amadeu Furtado,12901,8
2,Antônio Bezerra,28493,25
3,Barra do Ceará,79842,47
4,Barroso,32905,20
...,...,...,...
116,Planalto Ayrton Senna,43488,21
117,De Lourdes,3716,0
118,Olavo Oliveira,13403,4
119,Santa Maria,14709,9


In [71]:
df_final_bairros['equipamentos_por_habitantes'] = round(df_final_bairros['total']/df_final_bairros['pop_2010']*10000,2)

In [72]:
df_final_bairros.columns

Index(['nome', 'CAPS_x', 'CRAS', 'CREAS',
       'Centro de convivência e abrigo institucional', 'Outros_Ass_Social',
       'Unidade de Acolhimento Institucional para Crianças e Adolescentes',
       'qtd_ass_social', 'id', 'cod_ibge', 'cod_bairro', 'area_ha',
       'ser_atual', 'cod_regiao', 'The_geom', 'Base do SAMU', 'CAPS_y',
       'HOSPITAL', 'POLICLINICA', 'UAPS', 'UPA', 'qtd_saude',
       'Parque Escolar Estadual', 'Parque Escolar Municipal',
       'qtd_parque_escolar', 'Eqp FUNDURB', 'Praças', 'qtd_pracas_fundurb',
       'pop_2010', 'total', 'equipamentos_por_habitantes'],
      dtype='object')

In [73]:
df_final_bairros = df_final_bairros.sort_values(by='equipamentos_por_habitantes', ascending=False)[['The_geom','nome','ser_atual','equipamentos_por_habitantes','total','qtd_saude','qtd_ass_social','qtd_parque_escolar','qtd_pracas_fundurb','pop_2010']]

In [74]:
df_final_bairros.pop_2010.describe()

count     121.00
mean    22323.27
std     15210.91
min      1479.00
25%     11817.00
50%     17643.00
75%     30426.00
max     79842.00
Name: pop_2010, dtype: float64

In [75]:
resumo_ser = df_final_bairros.groupby('ser_atual').sum()
resumo_ser['equipamentos_por_habitantes'] = round(resumo_ser['total']/resumo_ser['pop_2010']*10000,2)

<ipython-input-75-bf3e5fce7a09>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  resumo_ser = df_final_bairros.groupby('ser_atual').sum()


In [76]:
resumo_ser = resumo_ser.reset_index()

In [77]:
resumo_ser['ser'] = resumo_ser.ser_atual.apply(lambda x: x.replace('Secretaria Executiva Regional', 'SER'))

In [78]:
resumo_ser = pd.merge(resumo_ser,pd.read_csv(path+'Secretarias_Executivas_Regionais.csv'), left_on='ser', right_on='regiao_adm')

In [79]:
resumo_ser.columns

Index(['ser_atual', 'equipamentos_por_habitantes', 'total', 'qtd_saude',
       'qtd_ass_social', 'qtd_parque_escolar', 'qtd_pracas_fundurb',
       'pop_2010', 'ser', 'FID', 'geom', 'id', 'regiao_adm', 'area_m2',
       'area_km2', 'area_ha', 'perimetr_m', 'ano_ref', 'fonte', 'epsg_codif'],
      dtype='object')

In [80]:
resumo_ser = resumo_ser[['ser_atual', 'equipamentos_por_habitantes', 'total', 'qtd_saude',
       'qtd_ass_social', 'qtd_parque_escolar', 'qtd_pracas_fundurb',
       'pop_2010', 'geom', 'regiao_adm', 'area_ha']]

In [81]:
resumo_ser['geom'] = resumo_ser.geom.apply(wkt.loads)

In [82]:
gdf_resumo_ser = gpd.GeoDataFrame(resumo_ser, geometry='geom', crs='EPSG:4326')
gdf_resumo_ser.to_file('gdf_resumo_ser.geojson', drive='GeoJSON')

In [83]:
gdf_bairros_final = gpd.GeoDataFrame(df_final_bairros, geometry='The_geom', crs='EPSG:4326')
gdf_bairros_final.to_file('gdf_bairros_final.geojson', drive='GeoJSON')

In [84]:
gdf_pontos = gpd.GeoDataFrame(df_pontos, geometry='The_geom', crs='EPSG:4326')
gdf_pontos.to_file('gdf_pontos.geojson', drive='GeoJSON')

# Dados relacionados à Ourtoga Onerosa:


In [85]:
arquivos = [
    'OODC_-_Outorga_Onerosa_do_Direito_de_Construir.csv',
    'Certificados_de_Potencial_Adicional_de_Construção_(CEPACs).csv',
    'Equipamentos_-_FUNDURB.csv',
    'Aplicações_por_Bairro_-_FUNDURB.csv',
    'OUC-_Operação_Urbana_Consorciada.csv']

### Outorga_Onerosa_do_Direito_de_Construir

In [86]:
oodc = pd.read_csv(path+arquivos[0])

In [87]:
oodc

,FID,id,The_geom,Nome,Tipo,Descrição,Bairro,cod_bairro,Regional,cod_regiao,Território,fonte,ano_ref,epsg_codif
0,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,16,POINT (-38.52947000354029 -3.751342021908112),Nossa Senhora de Fátima SPE,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nNossa Senhora de Fá...,Fátima,16,Secretaria Executiva Regional 4,V,15,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
1,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,14,POINT (-38.48509388242866 -3.759205525624139),M Aguiar Construções SPE,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nM Aguiar Construçõe...,Guararapes,60,Secretaria Executiva Regional 7,VII,24,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
2,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,17,POINT (-38.48943110986843 -3.759070532527772),Integral Justino Café Incorporações,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nIntegral Justino Ca...,Guararapes,60,Secretaria Executiva Regional 7,VII,24,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
3,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,18,POINT (-38.48777894375517 -3.765687741667834),Mota Machado & Oregon SPE XXXVIII Construções,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nMota Machado & Oreg...,Guararapes,60,Secretaria Executiva Regional 7,VII,24,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
4,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,12,POINT (-38.49192378771789 -3.75082048404004),JS 128 Empreendimentos,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nJS 128 Empreendimen...,São João do Tauape,35,Secretaria Executiva Regional 2,III,10,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,6,POINT (-38.491390231027 -3.770234648244746),Fibra Engenharia Eireli,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nFibra Engenharia Ei...,Luciano Cavalcante,58,Secretaria Executiva Regional 7,VII,24,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
100,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,8,POINT (-38.46881804580752 -3.747235438941766),JS 123 Empreendimentos,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nJS 123 Empreendimen...,Cocó,11,Secretaria Executiva Regional 7,VII,23,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
101,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,3,POINT (-38.48685346946783 -3.760223585145527),Engexata Engenharia,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nEngexata Engenharia...,Guararapes,60,Secretaria Executiva Regional 7,VII,24,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
102,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,7,POINT (-38.48526063282295 -3.760962639395849),José Wagner Teixeira Imóveis,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nJosé Wagner Teixeir...,Guararapes,60,Secretaria Executiva Regional 7,VII,24,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...


In [88]:
'Nº PROCESSO SEUMA'
'Nº PROCESSO FORTALEZA ONLINE'
'N° PROCESSO FORTALEZA ONLINE'
'N° PROCESSO SEUMA'
'Nº PROCESSO'
'REUNIÃO CPPD'
'Nº TERMO DE COMPROMISSO'

'Nº TERMO DE COMPROMISSO'

In [89]:
def trata_descricao(descricao):
  lista = descricao.replace('<b>N° PROCESSO SEUMA</b>','<b>N° PROCESSO SEUMA:</b>').replace('<p></p>','').replace('</b></br>','').replace('\n','').replace('\t','').replace('º','°')
  lista = lista.replace('N° PROCESSO SEUMA','N° PROCESSO').replace('N° PROCESSO FORTALEZA ONLINE','N° PROCESSO')
  lista = lista.split('<b>')[1:]
  novaDescricao = dict()
  for i in lista:
    div = i.split(':')
    novaDescricao[div[0]] = div[1]

  return novaDescricao

In [90]:
oodc['Descricao_dict'] = oodc['Descrição'].apply(lambda x: trata_descricao(x))

In [91]:
dados_descricao = oodc['Descricao_dict'].apply(pd.Series)[['REQUERENTE','N° ALVARÁ DE CONSTRUÇÃO','ATIVIDADE','ÍNDICE DE APROVEITAMENTO OUTORGADO','VALOR DA OUTORGA']]

In [92]:
oodc = pd.concat([oodc.drop(['Descricao_dict'], axis=1), dados_descricao], axis=1)

In [93]:
oodc['REQUERENTE'].value_counts()

Engexata Engenharia LTDA                                           2
Mota Machado & Oregon SPE XXIV Construções e Incorporações LTDA    2
Tâmara Construtora LTDA ME                                         2
José Wagner Teixeira Imóveis e Incorporações LTDA                  2
Construtora Ara LTDA                                               2
                                                                  ..
Francisco José Pontes                                              1
C. Rolim Engenharia LTDA                                           1
Marquise 8 Empreendimentos Imobiliários LTDA                       1
Associação de Poupança e Empréstimo Poupex                         1
Porto Freire Engenharia e Incorporação LTDA                        1
Name: REQUERENTE, Length: 98, dtype: int64

In [94]:
oodc.loc[oodc['VALOR DA OUTORGA'].str.contains('(outubro/2015)'),'VALOR DA OUTORGA'] = '176487,26'

<ipython-input-94-c45b9a642575>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  oodc.loc[oodc['VALOR DA OUTORGA'].str.contains('(outubro/2015)'),'VALOR DA OUTORGA'] = '176487,26'


In [95]:
oodc.loc[oodc['VALOR DA OUTORGA'].str.contains('agamento através do instrumento de Outorga Onerosa de Alteração de Us')]

,FID,id,The_geom,Nome,Tipo,Descrição,Bairro,cod_bairro,Regional,cod_regiao,Território,fonte,ano_ref,epsg_codif,REQUERENTE,N° ALVARÁ DE CONSTRUÇÃO,ATIVIDADE,ÍNDICE DE APROVEITAMENTO OUTORGADO,VALOR DA OUTORGA
38,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,70,POINT (-38.49797368279082 -3.732778425490295),Solar Praça Portugal Empreendimentos,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nSolar Praça Portuga...,Meireles,24,Secretaria Executiva Regional 2,III,7,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,Solar Praça Portugal Empreendimentos Imobiliár...,50,Residencial Multifamiliar (prédio de apartamen...,"0,85",Pagamento através do instrumento de Outorga O...
43,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,75,POINT (-38.48569285163559 -3.725154588365527),Construtora ARA,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nConstrutora ARA\n<p...,Mucuripe,26,Secretaria Executiva Regional 2,III,9,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,Construtora ARA,Ainda não foi emitido,Hotel Residência (apart-hotel e flat service),"0,95",Pagamento através do instrumento de Outorga O...
59,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,92,POINT (-38.48302109224152 -3.811612771780738),Pronefron Participações,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nPronefron Participa...,Cambeba,53,Secretaria Executiva Regional 6,VIII,28,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,Pronefron Participações LTDA,Ainda não foi emitido,"Clínica sem internamento (médica, odontológica...","0,20",Pagamento através do instrumento de Outorga O...
69,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,84,POINT (-38.51037229436933 -3.723593626088164),IA Empreendimentos,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nIA Empreendimentos ...,Meireles,24,Secretaria Executiva Regional 2,III,7,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,IA Empreendimentos Imobiliários LTDA,AC00001957/2020,Flats e apart-hoteis,"0,96",Pagamento através do instrumento de Outorga O...


In [96]:
oodc.loc[oodc['VALOR DA OUTORGA'].str.contains('agamento através do instrumento de Outorga Onerosa de Alteração de Us'),'VALOR DA OUTORGA'] = '0'

In [97]:
oodc.loc[oodc['VALOR DA OUTORGA'].str.contains('='),'VALOR DA OUTORGA'] = '302.208,17'

In [98]:
oodc.loc[oodc['VALOR DA OUTORGA'].str.contains('substituição dessa contrapartida financeira da OODC pela'),'VALOR DA OUTORGA'] = 'R$ 1.136.257,50'

In [99]:
oodc.loc[oodc['VALOR DA OUTORGA'].str.contains('Total'),'VALOR DA OUTORGA'] = '1.361.906,62'

In [100]:
oodc['valor_tratado'] = oodc['VALOR DA OUTORGA'].apply(lambda x: float(x.replace('R$','').replace('.','').replace(',','.')))

In [101]:
oodc.valor_tratado.sum()

29054105.4

In [102]:
# Valor por Bairro:
oodc.groupby('Bairro')['valor_tratado'].sum().reset_index().sort_values(by='valor_tratado', ascending = False)

,Bairro,valor_tratado
0,Aldeota,10713863.97
15,Meireles,4931552.57
5,Cocó,2957439.93
9,Guararapes,1536067.39
6,Dionísio Torres,1240069.42
17,Papicu,1093209.52
8,Fátima,964757.70
21,São João do Tauape,862627.68
2,Cambeba,801473.31
20,Sapiranga,751798.50


In [103]:
oodc.Nome.value_counts().head(10)

Idibra Participações                          3
Engexata Engenharia                           2
Mota Machado & Oregon SPE XXIV Construções    2
Tâmara Construtora                            2
José Wagner Teixeira Imóveis                  2
Construtora Ara                               2
Nossa Senhora de Fátima SPE                   1
KIC - Fortaleza SPE 3                         1
Construtora Columbia LTDA                     1
Colméia Living Garden Empreendimentos         1
Name: Nome, dtype: int64

In [104]:
oodc.loc[oodc.Nome == 'Idibra Participações']

,FID,id,The_geom,Nome,Tipo,Descrição,Bairro,cod_bairro,Regional,cod_regiao,Território,fonte,ano_ref,epsg_codif,REQUERENTE,N° ALVARÁ DE CONSTRUÇÃO,ATIVIDADE,ÍNDICE DE APROVEITAMENTO OUTORGADO,VALOR DA OUTORGA,valor_tratado
16,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,34,POINT (-38.47637100984262 -3.741954465597941),Idibra Participações,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nIdibra Participaçõe...,Papicu,27,Secretaria Executiva Regional 2,III,8,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,Idibra Participações LTDA,23298,Residencial Multifamiliar (prédio de apartamen...,"0,30","R$ 250.928,82",250928.82
25,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,45,POINT (-38.47639348337208 -3.742033960882789),Idibra Participações,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nIdibra Participaçõe...,Papicu,27,Secretaria Executiva Regional 2,III,8,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,Idibra Participações LTDA,23298,Residencial Multifamiliar (prédio de apartamen...,"0,10","R$ 83.642,94",83642.94
31,vw_OutorgaOnerosaDireitoConstruir.fid--16360be...,55,POINT (-38.48346924407625 -3.761636350522814),Idibra Participações,Mapeamento OODC,<b>REQUERENTE:</b></br>\t\nIdibra Participaçõe...,Guararapes,60,Secretaria Executiva Regional 7,VII,24,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,Idibra Participações S/A,230,Comércio e Serviços Mútiplos,"0,2","R$ 109.188,95",109188.95


In [105]:
oodc['ATIVIDADE'].value_counts()

Residencial Multifamiliar (prédio de apartamentos)                                                      72
Residencial Multifamiliar (prédio de apartamentos)                                                       8
Residencial Multifamiliar (unidades compactas)                                                           3
Comércio e Serviços Mútiplos                                                                             2
Residencial Multifamiliar (unidades compactas)                                                           2
Clínica sem internamento (médica, odontológica, psicológica, etc)                                        1
Conjunto Habitacional (grupo de prédio de apartamentos)                                                  1
Residencial Multifamiliar (prédio de aoartamentos)                                                       1
Residencial Multifamiliar (prédio de apartamentos) e Centro de Compras (comércio varejista em geral)     1
Residência Multifamiliar (prédio de a

In [106]:
bairros['oodc'] = bairros['cod_bairro'].map(oodc.cod_bairro.value_counts()).fillna(0).astype(int)
bairros.sort_values(by='oodc', ascending=False)[['nome','oodc']].head(10)

,nome,oodc
66,Luciano Cavalcante,17
69,Guararapes,12
22,Cocó,12
31,Fátima,10
13,Aldeota,8
34,Meireles,6
41,Papicu,6
48,São João do Tauape,4
46,Cambeba,4
26,Dionísio Torres,4


In [107]:
oodc.columns

Index(['FID', 'id', 'The_geom', 'Nome', 'Tipo', 'Descrição', 'Bairro',
       'cod_bairro', 'Regional', 'cod_regiao', 'Território', 'fonte',
       'ano_ref ', 'epsg_codif', 'REQUERENTE', 'N° ALVARÁ DE CONSTRUÇÃO',
       'ATIVIDADE', 'ÍNDICE DE APROVEITAMENTO OUTORGADO', 'VALOR DA OUTORGA',
       'valor_tratado'],
      dtype='object')

In [108]:
oodc['point'] = oodc['The_geom'].apply(wkt.loads)

In [109]:
oodc['lng'] = oodc['point'].apply(lambda x: round(x.x,4))
oodc['lat'] = oodc['point'].apply(lambda x: round(x.y,4))

Certificados_de_Potencial_Adicional_de_Construção_(CEPACs)

In [110]:
cepacs = pd.read_csv(path+arquivos[1])

In [111]:
cepacs

,FID,id,The_geom,Nome,Descrição,Bairro,cod_bairro,Regional,Território,Fonte,ano_ref,epsg_codif
0,vw_CEPACsEmitidos.fid--16360be6_188534013f0_10b7,1,POINT (-38.580432 -3.709631),CEPAC N° 01-2020 substitui CEPAC N° 01-2016,REQUERENTE:\t Rita Maria Furtado Cruz e Ana Ma...,Barra do Ceará,6,Secretaria Executiva Regional 1,3,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
1,vw_CEPACsEmitidos.fid--16360be6_188534013f0_10b8,2,POINT (-38.610709 -3.80019),CEPAC N° 02-2019 substitui CEPAC N° 01-2017,REQUERENTE:\t Fórum Brasil de Apoio e Intercâm...,Siqueira,90,Secretaria Executiva Regional 5,39,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
2,vw_CEPACsEmitidos.fid--16360be6_188534013f0_10b9,3,POINT (-38.490878 -3.750578),CEPAC N° 02-2017,REQUERENTE:\t Imobiliária Manhattan LTDA <br> ...,São João do Tauape,35,Secretaria Executiva Regional 2,10,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
3,vw_CEPACsEmitidos.fid--16360be6_188534013f0_10ba,4,POINT (-38.470062 -3.737576),CEPAC N° 02-2020 substitui CEPAC N° 03-2017,REQUERENTE:\t Craveiro Imobiliária LTDA <br> ...,Papicu,27,Secretaria Executiva Regional 2,8,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
4,vw_CEPACsEmitidos.fid--16360be6_188534013f0_10bb,5,POINT (-38.513206 -3.874505),CEPAC N° 01-2018,REQUERENTE:\t Invest Incorporação e Planejamen...,Pedras,69,Secretaria Executiva Regional 9,33,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
5,vw_CEPACsEmitidos.fid--16360be6_188534013f0_10bc,6,POINT (-38.583511 -3.836518),CEPAC N° 02-2018,REQUERENTE:\t João Américo da Silva <br> Nº P...,Aracapé,124,Secretaria Executiva Regional 10,34,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
6,vw_CEPACsEmitidos.fid--16360be6_188534013f0_10bd,7,POINT (-38.584531 -3.740125),CEPAC N° 01-2019,REQUERENTE:\t Romcy Admnistração de Imóveis LT...,Padre Andrade,47,Secretaria Executiva Regional 3,12,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
7,vw_CEPACsEmitidos.fid--16360be6_188534013f0_10be,8,POINT (-38.521317 -3.720173),CEPAC N° 03-2020,REQUERENTE:\t Boris Freres & Cia LTDA <br> Nº...,Centro,10,Secretaria Executiva Regional 12,1,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
8,vw_CEPACsEmitidos.fid--16360be6_188534013f0_10bf,9,POINT (-38.47903 -3.747213),CEPAC N° 04-2020,REQUERENTE:\t Alcimor Aguiar Rocha Neto <br> ...,Cocó,11,Secretaria Executiva Regional 7,23,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
9,vw_CEPACsEmitidos.fid--16360be6_188534013f0_10c0,10,POINT (-38.46837 -3.72916),CEPAC N° 05-2020,REQUERENTE:\t Sandras Restaurantes LTDA <br> ...,Vicente Pinzon,37,Secretaria Executiva Regional 2,9,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...


In [112]:
bairros['cepacs'] = bairros['cod_bairro'].map(cepacs.cod_bairro.value_counts()).fillna(0).astype(int)
bairros.sort_values(by='cepacs', ascending=False)[['nome','cepacs']].head(10)

,nome,cepacs
22,Cocó,3
17,Barra do Ceará,2
51,Vicente Pinzon,1
38,Pedras,1
41,Papicu,1
98,Siqueira,1
19,Centro,1
48,São João do Tauape,1
66,Luciano Cavalcante,1
91,Mondubim,1


### Equipamentos_-_FUNDURB

In [113]:
equipamentos_fundurb = pd.read_csv(path+arquivos[2])

In [114]:
equipamentos_fundurb

,FID,id,The_geom,Nome,Descrição,Tipologia,fonte,ano_ref,epsg_codif
0,vw_EquipamentosFundurb.fid--16360be6_188534013...,1,POINT (-38.585757 -3.804442),ARENINHA CAMPO SDO - NOVO MONDUBIM,NOME:\t ARENINHA CAMPO SDO - MONDUBIM/ ARENINH...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
1,vw_EquipamentosFundurb.fid--16360be6_188534013...,2,POINT (-38.551253 -3.774594),CAMPO DO ITAOCA,NOME:\t CAMPO SDO - ITAOCA/ ARENINHA CAMPO DO ...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
2,vw_EquipamentosFundurb.fid--16360be6_188534013...,3,POINT (-38.569294 -3.817568),CAMPO SÍTIO CÓRREGO - MONDUBIM,NOME:\t CAMPO SÍTIO CÓRREGO - MONDUBIM <br> Nº...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
3,vw_EquipamentosFundurb.fid--16360be6_188534013...,4,POINT (-38.563478 -3.78356),CAMPO DO REAL/ CAMPO DO VILA BETÂNIA,NOME:\t CAMPO DO REAL/ CAMPO DO VILA BETÂNIA <...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
4,vw_EquipamentosFundurb.fid--16360be6_188534013...,5,POINT (-38.593219 -3.782133),PRAÇA BONSUCESSO / CAMPINHO,NOME:\t PRAÇA BONSUCESSO / CAMPINHO BONSUCESSO...,Praças,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
5,vw_EquipamentosFundurb.fid--16360be6_188534013...,6,POINT (-38.545936 -3.750987),PRAÇA FREI GALVÃO,NOME:\t PRAÇA FREI GALVÃO <br> Nº CONTRATO: 51...,Praças,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
6,vw_EquipamentosFundurb.fid--16360be6_188534013...,7,POINT (-38.556568 -3.835197),CAMPINHO JOSÉ WALTER,NOME:\t CAMPINHO JOSÉ WALTER <br> Nº CONTRATO:...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
7,vw_EquipamentosFundurb.fid--16360be6_188534013...,8,POINT (-38.568015 -3.756057),CAMPINHO MAUÁ,NOME:\t CAMPINHO MAUÁ <br> Nº CONTRATO: 51/201...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
8,vw_EquipamentosFundurb.fid--16360be6_188534013...,9,POINT (-38.572659 -3.767755),PRAÇA NEY REBOUÇAS,NOME:\t PRAÇA NEY REBOUÇAS <br> Nº CONTRATO: 5...,Praças,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
9,vw_EquipamentosFundurb.fid--16360be6_188534013...,10,POINT (-38.595083 -3.744294),CAMPINHO IPIRANGA,NOME:\t CAMPINHO IPIRANGA <br> Nº CONTRATO: 51...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...


In [115]:
equipamentos_fundurb.iloc[0]['Descrição']

'NOME:\t ARENINHA CAMPO SDO - MONDUBIM/ ARENINHA CAMPO SDO - NOVO MONDUBIM <br> Nº CONTRATO: 22/2018; 32/2019 <br> VALOR TOTAL DA OBRA: R$ 1.023.240,84 <br> REPASSE DO FUNDURB: R$ adicionar <br> INÍCIO DA OBRA: 04/06/2019 <br> STATUS: Inaugurada em 11/01'

In [116]:
equipamentos_fundurb.loc[equipamentos_fundurb['Descrição'].str.contains(' REPASSE DO FUNDURB::')]

,FID,id,The_geom,Nome,Descrição,Tipologia,fonte,ano_ref,epsg_codif


In [117]:
equipamentos_fundurb.loc[equipamentos_fundurb['Descrição'].str.contains('VALOR TOTAL DA OBRA:')].shape

(34, 9)

In [118]:
equipamentos_fundurb.loc[equipamentos_fundurb['Descrição'].str.contains('REPASSE DO FUNDURB:')].shape

(34, 9)

In [119]:
equipamentos_fundurb['repasse'] = equipamentos_fundurb['Descrição'].apply(lambda x: x.split('REPASSE DO FUNDURB:')[1].split('<br>')[0] if len(x.split('REPASSE DO FUNDURB:'))>1 else 'inv')

In [120]:
equipamentos_fundurb.loc[equipamentos_fundurb['repasse'].str.contains('adicionar')].shape

(6, 10)

In [121]:
equipamentos_fundurb.loc[(~equipamentos_fundurb['repasse'].str.contains('adicionar')) & (~equipamentos_fundurb['repasse'].str.contains('inv'))]['repasse'].apply(lambda x: float(x.replace('R$','').replace('.','').replace(',','.'))).sum()

14846738.790000001

In [122]:
list(equipamentos_fundurb.loc[equipamentos_fundurb['repasse'].str.contains('adicionar')]['Nome'])

['ARENINHA CAMPO SDO - NOVO MONDUBIM',
 'CAMPO DO ITAOCA',
 'CAMPO SÍTIO CÓRREGO - MONDUBIM',
 'PRAÇA BONSUCESSO / CAMPINHO',
 'CAMPINHO IPIRANGA',
 'ESTÁDIO ANTONY COSTA - ANTONIO BEZERRA']

In [123]:
equipamentos_fundurb.columns

Index(['FID', 'id', 'The_geom', 'Nome', 'Descrição', 'Tipologia', 'fonte',
       'ano_ref', 'epsg_codif', 'repasse'],
      dtype='object')

In [124]:
equipamentos_fundurb['The_geom'] = equipamentos_fundurb.The_geom.apply(wkt.loads)

In [125]:
equipamentos_fundurb['lng'] = equipamentos_fundurb.The_geom.apply(lambda x: x.x)
equipamentos_fundurb['lat'] = equipamentos_fundurb.The_geom.apply(lambda x: x.y)

In [126]:
equipamentos_fundurb

,FID,id,The_geom,Nome,Descrição,Tipologia,fonte,ano_ref,epsg_codif,repasse,lng,lat
0,vw_EquipamentosFundurb.fid--16360be6_188534013...,1,POINT (-38.585757 -3.804442),ARENINHA CAMPO SDO - NOVO MONDUBIM,NOME:\t ARENINHA CAMPO SDO - MONDUBIM/ ARENINH...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,R$ adicionar,-38.59,-3.80
1,vw_EquipamentosFundurb.fid--16360be6_188534013...,2,POINT (-38.551253 -3.774594),CAMPO DO ITAOCA,NOME:\t CAMPO SDO - ITAOCA/ ARENINHA CAMPO DO ...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,adicionar,-38.55,-3.77
2,vw_EquipamentosFundurb.fid--16360be6_188534013...,3,POINT (-38.569294 -3.817568),CAMPO SÍTIO CÓRREGO - MONDUBIM,NOME:\t CAMPO SÍTIO CÓRREGO - MONDUBIM <br> Nº...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,adicionar,-38.57,-3.82
3,vw_EquipamentosFundurb.fid--16360be6_188534013...,4,POINT (-38.563478 -3.78356),CAMPO DO REAL/ CAMPO DO VILA BETÂNIA,NOME:\t CAMPO DO REAL/ CAMPO DO VILA BETÂNIA <...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 1.866.817,21",-38.56,-3.78
4,vw_EquipamentosFundurb.fid--16360be6_188534013...,5,POINT (-38.593219 -3.782133),PRAÇA BONSUCESSO / CAMPINHO,NOME:\t PRAÇA BONSUCESSO / CAMPINHO BONSUCESSO...,Praças,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,adicionar,-38.59,-3.78
5,vw_EquipamentosFundurb.fid--16360be6_188534013...,6,POINT (-38.545936 -3.750987),PRAÇA FREI GALVÃO,NOME:\t PRAÇA FREI GALVÃO <br> Nº CONTRATO: 51...,Praças,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 1.597,67",-38.55,-3.75
6,vw_EquipamentosFundurb.fid--16360be6_188534013...,7,POINT (-38.556568 -3.835197),CAMPINHO JOSÉ WALTER,NOME:\t CAMPINHO JOSÉ WALTER <br> Nº CONTRATO:...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 18.664,28",-38.56,-3.84
7,vw_EquipamentosFundurb.fid--16360be6_188534013...,8,POINT (-38.568015 -3.756057),CAMPINHO MAUÁ,NOME:\t CAMPINHO MAUÁ <br> Nº CONTRATO: 51/201...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 108.565,44",-38.57,-3.76
8,vw_EquipamentosFundurb.fid--16360be6_188534013...,9,POINT (-38.572659 -3.767755),PRAÇA NEY REBOUÇAS,NOME:\t PRAÇA NEY REBOUÇAS <br> Nº CONTRATO: 5...,Praças,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 90.393,95",-38.57,-3.77
9,vw_EquipamentosFundurb.fid--16360be6_188534013...,10,POINT (-38.595083 -3.744294),CAMPINHO IPIRANGA,NOME:\t CAMPINHO IPIRANGA <br> Nº CONTRATO: 51...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,adicionar,-38.60,-3.74


In [127]:
equipamentos_fundurb['valor'] = equipamentos_fundurb.loc[equipamentos_fundurb['Descrição'].str.contains('VALOR TOTAL DA OBRA:')]['Descrição'].apply(lambda x: x.split('VALOR TOTAL DA OBRA:')[1].split('<br>')[0])

In [128]:
equipamentos_fundurb['valor_tratado'] = equipamentos_fundurb.valor.apply(lambda x: float((str(x)).replace("R$","").replace(".","").replace(",",".")))

In [129]:
equipamentos_fundurb['valor_tratado'].describe()

count        34.00
mean     876162.88
std     1177999.32
min        1597.67
25%      207512.40
50%      636772.39
75%     1058953.60
max     6735593.01
Name: valor_tratado, dtype: float64

In [130]:
equipamentos_fundurb.sort_values(by="valor_tratado", ascending=False)

,FID,id,The_geom,Nome,Descrição,Tipologia,fonte,ano_ref,epsg_codif,repasse,lng,lat,valor,valor_tratado
3,vw_EquipamentosFundurb.fid--16360be6_188534013...,4,POINT (-38.563478 -3.78356),CAMPO DO REAL/ CAMPO DO VILA BETÂNIA,NOME:\t CAMPO DO REAL/ CAMPO DO VILA BETÂNIA <...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 1.866.817,21",-38.56,-3.78,"R$ 6.735.593,01",6735593.01
1,vw_EquipamentosFundurb.fid--16360be6_188534013...,2,POINT (-38.551253 -3.774594),CAMPO DO ITAOCA,NOME:\t CAMPO SDO - ITAOCA/ ARENINHA CAMPO DO ...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,adicionar,-38.55,-3.77,"R$ 1.942.942,17",1942942.17
21,vw_EquipamentosFundurb.fid--16360be6_188534013...,22,POINT (-38.590367 -3.74455),ESTÁDIO ANTONY COSTA - ANTONIO BEZERRA,NOME:\t ESTÁDIO ANTONY COSTA - ANTONIO BEZERRA...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,adicionar,-38.59,-3.74,"R$ 1.840.805,52",1840805.52
22,vw_EquipamentosFundurb.fid--16360be6_188534013...,23,POINT (-38.535673 -3.791637),CAMPO DO SERRINHA - SERRINHA,NOME:\t CAMPO DO SERRINHA - SERRINHA <br> Nº C...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 1.514.444,26",-38.54,-3.79,"R$ 1.794.068,34",1794068.34
18,vw_EquipamentosFundurb.fid--16360be6_188534013...,19,POINT (-38.60048 -3.777161),PRAÇA DA JUVENTUDE - GRANJA PORTUGAL,NOME:\t PRAÇA DA JUVENTUDE - GRANJA PORTUGAL <...,Praças,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 1.125.370,78",-38.60,-3.78,"R$ 1.694.959,50",1694959.50
20,vw_EquipamentosFundurb.fid--16360be6_188534013...,21,POINT (-38.488008 -3.856663),CAMPO DO ALTO ALEGRE - SÃO BENTO,NOME:\t CAMPO DO ALTO ALEGRE - SÃO BENTO <br> ...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 1.217.506,15",-38.49,-3.86,"R$ 1.277.011,41",1277011.41
19,vw_EquipamentosFundurb.fid--16360be6_188534013...,20,POINT (-38.508668 -3.81452),CAMPO DO VIOLETA - BARROSO,NOME:\t CAMPO DO VIOLETA - BARROSO <br> Nº CON...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 204.443,75",-38.51,-3.81,"R$ 1.127.577,60",1127577.60
31,vw_EquipamentosFundurb.fid--16360be6_188534013...,32,POINT (-38.546045 -3.741906),CAMPO DO TIGRÃO - RODOLFO TEÓFILO,NOME:\t CAMPO DO TIGRÃO - RODOLFO TEÓFILO <br>...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 902.956,89",-38.55,-3.74,"R$ 1.120.338,77",1120338.77
29,vw_EquipamentosFundurb.fid--16360be6_188534013...,30,POINT (-38.547251 -3.804527),CAMPO SDO - PARQUE DOIS IRMÃOS,NOME:\t CAMPO SDO - PARQUE DOIS IRMÃOS <br> Nº...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,"R$ 683.394,49",-38.55,-3.80,"R$ 1.070.857,86",1070857.86
0,vw_EquipamentosFundurb.fid--16360be6_188534013...,1,POINT (-38.585757 -3.804442),ARENINHA CAMPO SDO - NOVO MONDUBIM,NOME:\t ARENINHA CAMPO SDO - MONDUBIM/ ARENINH...,Areninhas,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,R$ adicionar,-38.59,-3.80,"R$ 1.023.240,84",1023240.84


In [131]:
gdf_equipamentos_fundurb = gpd.GeoDataFrame(equipamentos_fundurb, geometry='The_geom')

In [132]:
gdf_equipamentos_fundurb.to_file('gdf_equipamentos_fundurb.geojson',drive='GeoJson')

In [133]:
equipamentos_fundurb.loc[~equipamentos_fundurb['Descrição'].str.contains('VALOR TOTAL DA OBRA:')]

,FID,id,The_geom,Nome,Descrição,Tipologia,fonte,ano_ref,epsg_codif,repasse,lng,lat,valor,valor_tratado
34,vw_EquipamentosFundurb.fid--16360be6_188534013...,35,POINT (-38.605518 -3.809822),EQUIPAMENTO CULTURAL,NOME: CENTRO CULTURAL CANINDEZIN...,Equipamento Cultural,SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,inv,-38.61,-3.81,NaN,NaN


In [134]:
equipamentos_fundurb.loc[~equipamentos_fundurb['valor'].isna()]['valor'].apply(lambda x: float(x.replace('R$','').replace('.','').replace(',','.'))).sum()

29789537.93

### Aplicações_por_Bairro_-_FUNDURB

In [135]:
aplicacoes_fundurb = pd.read_csv(path+arquivos[3])

In [136]:
aplicacoes_fundurb.columns

Index(['FID', 'id', 'The_geom', 'Nome', 'Descrição', 'Aplicações', 'fonte',
       'ano_ref', 'epsg_codif'],
      dtype='object')

In [137]:
aplicacoes_fundurb[['id','Nome','Descrição','Aplicações','ano_ref']]

,id,Nome,Descrição,Aplicações,ano_ref
0,1,Praia do Futuro I e II,SERVIÇOS / ORÇAMENTO <br><br> ASFALTO\tR$ 7.88...,"1.000.000,00 - 1.933.302,49",2023
1,2,Messejana\n,SERVIÇOS / ORÇAMENTO<br> <br> ASFALTO\tR$ 71.0...,"1.000.000,00 - 1.933.302,49",2023
2,3,Parquelândia\n,SERVIÇOS / ORÇAMENTO <br><br> ASFALTO\tR$ 20.9...,"10.000,00 - 50.000,00",2023
3,4,Jardim Iracema\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 8.654,17...","10.000,00 - 50.000,00",2023
4,5,Amadeu Furtado\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 11.105,6...","10.000,00 - 50.000,00",2023
...,...,...,...,...,...
90,94,Farias Brito\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 69.285,1...","50.000,00 - 100.000,00",2023
91,95,Jardim Guanabara\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 56.995,9...","50.000,00 - 100.000,00",2023
92,91,Cristo Redentor\n,SERVIÇOS / ORÇAMENTO<br> <br> ASFALTO\tR$ 83.4...,"50.000,00 - 100.000,00",2023
93,92,Papicu\n,SERVIÇOS / ORÇAMENTO <br><br> ASFALTO\tR$ 92.3...,"50.000,00 - 100.000,00",2023


In [138]:
aplicacoes_fundurb.loc[aplicacoes_fundurb.Nome.str.contains('Aldeota')].iloc[0]['Descrição']

'SERVIÇOS / ORÇAMENTO <br> <br>ASFALTO\tR$ 42.008,44 <br> <br>CALÇAMENTO\tR$ 39.441,34 <br> <br>PASSEIOS\tR$ 1.817,15 <br> <br>SERVIÇOS DIVERSOS - (ESCAVAÇÕES, DEMOLIÇÕES, CARGA E TRANSPORTE DE MATERIAIS E TERRAPLENAGEM)\tR$ 48.982,82 <br> <br>TOTAL\tR'

In [139]:
aplicacoes_fundurb.loc[aplicacoes_fundurb.Nome.str.contains('Aldeota')].iloc[0]['Aplicações']

'100.000,00 - 1.000.000,00'

In [140]:
aplicacoes_fundurb.loc[aplicacoes_fundurb.Nome.str.contains('Meireles')].iloc[0]['Descrição']

'SERVIÇOS / ORÇAMENTO <br><br> ASFALTO\tR$ 1.021,75 <br> <br>DRENAGEM\tR$ 39,06 <br> <br>SERVIÇOS DIVERSOS - (ESCAVAÇÕES, DEMOLIÇÕES, CARGA E TRANSPORTE DE MATERIAIS E TERRAPLENAGEM)\tR$ 304,18 <br> <br>TOTAL\tR$ 1.364,99'

In [141]:
aplicacoes_fundurb.loc[aplicacoes_fundurb.Nome.str.contains('Meireles')].iloc[0]['Aplicações']

'124,36 - 10.000,00'

In [142]:
aplicacoes_fundurb.loc[aplicacoes_fundurb.Nome.str.contains('Cocó')].iloc[0]['Descrição']

'SERVIÇOS / ORÇAMENTO <br><br> ASFALTO\tR$ 15.113,42 <br> <br>CALÇAMENTO\tR$ 15,99 <br> <br>PASSEIOS\tR$ 169,02 <br> <br>SERVIÇOS DIVERSOS - (ESCAVAÇÕES, DEMOLIÇÕES, CARGA E TRANSPORTE DE MATERIAIS E TERRAPLENAGEM)\tR$ 147,83 <br> <br>TOTAL\tR$ 15.446,'

In [143]:
aplicacoes_fundurb.loc[aplicacoes_fundurb.Nome.str.contains('Cocó')].iloc[0]['Aplicações']

'10.000,00 - 50.000,00'

In [144]:
aplicacoes_com_total = aplicacoes_fundurb.loc[aplicacoes_fundurb['Descrição'].str.contains('TOTAL')]

In [145]:
aplicacoes_com_total['desc'] = aplicacoes_com_total['Descrição'].apply(lambda x: x.split('TOTAL')[1].replace('\t','').replace('R$','').replace('.','').replace(',','.'))

<ipython-input-145-6d53f8ed5193>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aplicacoes_com_total['desc'] = aplicacoes_com_total['Descrição'].apply(lambda x: x.split('TOTAL')[1].replace('\t','').replace('R$','').replace('.','').replace(',','.'))


In [146]:
aplicacoes_com_total.loc[aplicacoes_com_total.desc.str.contains('R')].iloc[1]['Descrição']

'SERVIÇOS / ORÇAMENTO <br> <br>ASFALTO\tR$ 42.008,44 <br> <br>CALÇAMENTO\tR$ 39.441,34 <br> <br>PASSEIOS\tR$ 1.817,15 <br> <br>SERVIÇOS DIVERSOS - (ESCAVAÇÕES, DEMOLIÇÕES, CARGA E TRANSPORTE DE MATERIAIS E TERRAPLENAGEM)\tR$ 48.982,82 <br> <br>TOTAL\tR'

In [147]:
# ['Descrição'].apply(lambda x: float(x.split('TOTAL')[1].replace('\t','').replace('R$','').replace('.','').replace(',','.')))

In [148]:
aplicacoes_fundurb.loc[aplicacoes_fundurb['Descrição'].str.contains('TOTAL')].iloc[0]['Descrição']

'SERVIÇOS / ORÇAMENTO <br><br> ASFALTO\tR$ 20.944,81 <br> <br>DRENAGEM\tR$ 497,42 <br> <br>PASSEIOS\tR$ 227,38 <br> <br>SERVIÇOS DIVERSOS - (ESCAVAÇÕES, DEMOLIÇÕES, CARGA E TRANSPORTE DE MATERIAIS E TERRAPLENAGEM)\tR$ 477,93 <br> <br>TOTAL\tR$ 22.147,54'

In [149]:
aplicacoes_fundurb.loc[aplicacoes_fundurb['Descrição'].str.contains('TOTAL')]

,FID,id,The_geom,Nome,Descrição,Aplicações,fonte,ano_ref,epsg_codif
2,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,3,"MULTIPOLYGON (((-38.564178 -3.735583, -38.5598...",Parquelândia\n,SERVIÇOS / ORÇAMENTO <br><br> ASFALTO\tR$ 20.9...,"10.000,00 - 50.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
3,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,4,"MULTIPOLYGON (((-38.588664 -3.714560999999999,...",Jardim Iracema\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 8.654,17...","10.000,00 - 50.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
4,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,5,"MULTIPOLYGON (((-38.552082 -3.738988, -38.5519...",Amadeu Furtado\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 11.105,6...","10.000,00 - 50.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
5,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,6,"MULTIPOLYGON (((-38.543786 -3.737278, -38.5438...",Parque Araxá\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 30.149,8...","10.000,00 - 50.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
6,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,7,"MULTIPOLYGON (((-38.470815 -3.745124, -38.4688...",Cocó\n,SERVIÇOS / ORÇAMENTO <br><br> ASFALTO\tR$ 15.1...,"10.000,00 - 50.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
...,...,...,...,...,...,...,...,...,...
89,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,90,"MULTIPOLYGON (((-38.584964 -3.744534, -38.5849...",Dom Lustosa\n,SERVIÇOS / ORÇAMENTO <br> <br>ASFALTO\tR$ 4.05...,"50.000,00 - 100.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
90,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,94,"MULTIPOLYGON (((-38.545667 -3.726659, -38.5452...",Farias Brito\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 69.285,1...","50.000,00 - 100.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
91,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,95,"MULTIPOLYGON (((-38.587102 -3.720637, -38.5867...",Jardim Guanabara\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 56.995,9...","50.000,00 - 100.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
92,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,91,"MULTIPOLYGON (((-38.568969 -3.702138, -38.5689...",Cristo Redentor\n,SERVIÇOS / ORÇAMENTO<br> <br> ASFALTO\tR$ 83.4...,"50.000,00 - 100.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...


In [150]:
aplicacoes_fundurb

,FID,id,The_geom,Nome,Descrição,Aplicações,fonte,ano_ref,epsg_codif
0,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,1,"MULTIPOLYGON (((-38.456538 -3.730468, -38.4561...",Praia do Futuro I e II,SERVIÇOS / ORÇAMENTO <br><br> ASFALTO\tR$ 7.88...,"1.000.000,00 - 1.933.302,49",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
1,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,2,"MULTIPOLYGON (((-38.492775 -3.811155, -38.4927...",Messejana\n,SERVIÇOS / ORÇAMENTO<br> <br> ASFALTO\tR$ 71.0...,"1.000.000,00 - 1.933.302,49",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
2,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,3,"MULTIPOLYGON (((-38.564178 -3.735583, -38.5598...",Parquelândia\n,SERVIÇOS / ORÇAMENTO <br><br> ASFALTO\tR$ 20.9...,"10.000,00 - 50.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
3,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,4,"MULTIPOLYGON (((-38.588664 -3.714560999999999,...",Jardim Iracema\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 8.654,17...","10.000,00 - 50.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
4,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,5,"MULTIPOLYGON (((-38.552082 -3.738988, -38.5519...",Amadeu Furtado\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 11.105,6...","10.000,00 - 50.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
...,...,...,...,...,...,...,...,...,...
90,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,94,"MULTIPOLYGON (((-38.545667 -3.726659, -38.5452...",Farias Brito\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 69.285,1...","50.000,00 - 100.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
91,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,95,"MULTIPOLYGON (((-38.587102 -3.720637, -38.5867...",Jardim Guanabara\n,"SERVIÇOS / ORÇAMENTO <br> ASFALTO\tR$ 56.995,9...","50.000,00 - 100.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
92,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,91,"MULTIPOLYGON (((-38.568969 -3.702138, -38.5689...",Cristo Redentor\n,SERVIÇOS / ORÇAMENTO<br> <br> ASFALTO\tR$ 83.4...,"50.000,00 - 100.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...
93,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,92,"MULTIPOLYGON (((-38.466709 -3.746442, -38.4688...",Papicu\n,SERVIÇOS / ORÇAMENTO <br><br> ASFALTO\tR$ 92.3...,"50.000,00 - 100.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...


In [151]:
aplicacoes_fundurb['nome'] = aplicacoes_fundurb.Nome.str.replace('\n','')

In [152]:
bairros.head(1)

,id,nome,cod_ibge,cod_bairro,area_ha,ser_atual,cod_regiao,The_geom,oodc,cepacs
0,77,Rachel de Queiroz,2304400077,77,196.88,Secretaria Executiva Regional 8,VI,MULTIPOLYGON (((-38.554614978346514 -3.8094727...,0,0


In [153]:
#Algum bairro aparece repetido?
aplicacoes_fundurb.nome.value_counts()

Praia do Futuro I e II    1
Cajazeiras                1
Parque Dois Irmãos        1
Aerolândia                1
Edson Queiroz             1
                         ..
Bom Jardim                1
Carlito Pamplona          1
Vila Velha                1
Parque Iracema            1
Cais do Porto             1
Name: nome, Length: 95, dtype: int64

In [154]:
aplicacoes_fundurb.loc[aplicacoes_fundurb.nome == 'Cidade 2.000','nome'] = 'Cidade 2000'

In [155]:
teste_merge  = pd.merge(bairros,aplicacoes_fundurb, on='nome', how='outer', indicator =True)

In [156]:
teste_merge._merge.value_counts()

both          86
left_only     35
right_only     9
Name: _merge, dtype: int64

In [157]:
teste_merge.loc[teste_merge._merge == 'right_only']['nome']

121    Praia do Futuro I e II
122                   Guajeru
123                 Vila Pery
124              Álvaro Weyne
125        Presidente Keneddy
126            Conjunto Ceará
127             Manoel Sátiro
128      Prefeito José Walter
129               Vila Ellery
Name: nome, dtype: object

In [158]:
bairros_que_faltam = ['Rachel de Queiroz', 'Bom Futuro', 'São Gerardo',
 'Parreão',
 'Pedras','Barra do Ceará',
 'Cambeba',
 'Varjota',
 'Jardim Cearense',
 'Quintino Cunha',
 'Curió',
 'Guararapes',
 'Sapiranga',
 'Sabiaguaba',
 'Salinas',
 'Maraponga',
 'Parque Santa Rosa',
 'Presidente Vargas',
 'Aeroporto',
 'Manuel Dias Branco',
 'Olavo Oliveira',
 'Santa Maria',
 'São Bento',
 'Novo Mondubim']
bairros_que_faltam = bairros.loc[bairros.nome.isin(bairros_que_faltam)][['The_geom','nome']]
bairros_que_faltam['Descrição'] = 'O Bairro não aparece na base de aplicações FUNDURB'
bairros_que_faltam['Total'] = 0
bairros_que_faltam.to_csv('bairros_que_faltam.csv', index=False)

In [159]:
teste_merge.loc[teste_merge._merge == 'left_only']['nome'].to_list()

['Rachel de Queiroz',
 'Bom Futuro',
 'São Gerardo',
 'Alvaro Weyne',
 'Barra do Ceará',
 'Parreão',
 'Pedras',
 'Conjunto Ceará 1',
 'Presidente Kennedy',
 'Cambeba',
 'Varjota',
 'Bairro Ellery',
 'Jardim Cearense',
 'Quintino Cunha',
 'Curió',
 'Guajerú',
 'Guararapes',
 'Vila Manuel Sátiro',
 'Sapiranga',
 'Sabiaguaba',
 'Salinas',
 'Conjunto Ceará 2',
 'Maraponga',
 'Parque Santa Rosa',
 'Presidente Vargas',
 'José Walter',
 'Aeroporto',
 'Vila Peri',
 'Praia do Futuro I',
 'Manuel Dias Branco',
 'Praia do Futuro II',
 'Olavo Oliveira',
 'Santa Maria',
 'São Bento',
 'Novo Mondubim']

In [160]:
atualiza_nomes = {
      'Cidade 2.000':'Cidade 2000',
      'Vila Ellery':'Bairro Ellery',
      'Prefeito José Walter':'José Walter',
      'Manoel Sátiro':'Vila Manuel Sátiro',
      'Presidente Keneddy':'Presidente Kennedy',
      'Álvaro Weyne':'Alvaro Weyne',
      'Vila Pery':'Vila Peri',
      'Guajeru':'Guajerú'
    }

In [161]:
bairros.loc[bairros.nome.str.contains('Guaj')]

,id,nome,cod_ibge,cod_bairro,area_ha,ser_atual,cod_regiao,The_geom,oodc,cepacs
67,59,Guajerú,2304400059,59,107.40,Secretaria Executiva Regional 6,VIII,MULTIPOLYGON (((-38.47443914274054 -3.82629562...,0,0


In [162]:
aplicacoes_fundurb.loc[aplicacoes_fundurb.nome == 'Vila Ellery']

,FID,id,The_geom,Nome,Descrição,Aplicações,fonte,ano_ref,epsg_codif,nome
65,vw_FundurbAplicacoes.fid--16360be6_188534013f0...,71,"MULTIPOLYGON (((-38.567147 -3.720869, -38.5668...",Vila Ellery\n,"SERVIÇOS / ORÇAMENTO <br> CALÇAMENTO\tR$ 101,3...","124,36 - 10.000,00",SEUMA,2023,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,Vila Ellery


In [163]:
bairros.loc[bairros.nome.str.contains('Praia do F')]

,id,nome,cod_ibge,cod_bairro,area_ha,ser_atual,cod_regiao,The_geom,oodc,cepacs
109,111,Praia do Futuro I,2304400113,113,173.32,Secretaria Executiva Regional 7,VII,MULTIPOLYGON (((-38.45635947007637 -3.73028757...,0,0
113,112,Praia do Futuro II,2304400114,114,316.56,Secretaria Executiva Regional 7,VII,MULTIPOLYGON (((-38.447929090376256 -3.7470713...,0,0


### OUC-_Operação_Urbana_Consorciada

In [164]:
ouc =  pd.read_csv(path+arquivos[4])

In [165]:
ouc

,FID,id,The_geom,sigla,nome,area_m2,perimetro_m,legislacao,obs,fonte,epsg_codif,ano_ref
0,vw_OperacaoUrbanaConsorciada.fid--16360be6_188...,3,MULTIPOLYGON (((-38.576407728579966 -3.7651828...,OUC,OPERAÇÃO URBANA CONSORCIADA JOCKEY CLUBE,228738,1961.77,"LEI N 9333, DE 28 DE DEZEMBRO DE 2007",PUBLICADA NO DOM 13.745 DE 24/01/2008,SEUMA,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,2022
1,vw_OperacaoUrbanaConsorciada.fid--16360be6_188...,1,MULTIPOLYGON (((-38.484143484208246 -3.7233213...,OUC,OPERAÇÃO URBANA CONSORCIADA PARQUE FOZ DO RIAC...,30270,824.52,"LEI N 8503, DE 26 DE DEZEMBRO DE 2000",PUBLICADA NO DOM 12.267 DE 04/02/2002,SEUMA,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,2022
2,vw_OperacaoUrbanaConsorciada.fid--16360be6_188...,2,MULTIPOLYGON (((-38.44838920107681 -3.76398569...,OUC,OPERAÇÃO URBANA CONSORCIADA DUNAS DO COCÓ,3354726,10422.88,"LEI N 8915, DE 23 DE DEZEMBRO DE 2004",PUBLICADA NO DOM 12.983 DE 23/12/2004,SEUMA,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,2022
3,vw_OperacaoUrbanaConsorciada.fid--16360be6_188...,6,MULTIPOLYGON (((-38.46892507012013 -3.74312614...,OUC,OPERAÇÃO URBANA CONSORCIADA LAGOA DO PAPICU - ...,133942,1652.95,LEI Nº 9857 DE 22 DE DEZEMBRO DE 2011,PUBLICADA NO DOM 14.808 DE 08/06/2012,SEUMA,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,2022
4,vw_OperacaoUrbanaConsorciada.fid--16360be6_188...,4,MULTIPOLYGON (((-38.4925628098341 -3.775648693...,OUC,OPERAÇÃO URBANA CONSORCIADA SITIO TUNGA,247019,2914.64,"LEI Nº 9778, DE 24 DE MAIO DE 2011",PUBLICADA NO DOM 14.592 DE 19/07/2011,SEUMA,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,2022
5,vw_OperacaoUrbanaConsorciada.fid--16360be6_188...,5,MULTIPOLYGON (((-38.47379235137911 -3.73943351...,OUC,OPERAÇÃO URBANA CONSORCIADA LAGOA DO PAPICU - ...,82830,1344.17,LEI Nº 9857 DE 22 DE DEZEMBRO DE 2011,PUBLICADA NO DOM 14.808 DE 08/06/2012,SEUMA,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,2022
6,vw_OperacaoUrbanaConsorciada.fid--16360be6_188...,8,MULTIPOLYGON (((-38.46556502368446 -3.80418064...,OUC,OPERAÇÃO URBANA CONSORCIADA PARQUE URBANO DA L...,878375,4497.46,"LEI Nº 10.404, DE 13 DE OUTUBRO DE 2015",PUBLICADA NO DOM 14.592 DE 19/10/2015,SEUMA,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,2022
7,vw_OperacaoUrbanaConsorciada.fid--16360be6_188...,7,MULTIPOLYGON (((-38.607428724169 -3.8156131330...,OUC,OPERAÇÃO URBANA CONSORCIADA OSÓRIO DE PAIVA,1798845,8813.04,"LEI Nº 10.403, DE 13 DE OUTUBRO DE 2015",PUBLICADA NO DOM 14.592 DE 19/10/2015,SEUMA,EPSG: 31984 (SIRGAS 2000 - UTM 24S); Codificaç...,2022


### OOAU - Outorga Onerosa Alteração de Uso

In [166]:
ooau = pd.read_csv(f'{path}/OOAU.csv').iloc[:25]

In [167]:
gmaps = googlemaps.Client(key='chave-google-maps-api')

In [168]:
ooau

,Endereço,Mês/ano,Valor da OOAU,Termo de compromisso,Pagamento,O que outorgou,Altura proposta,Observações
0,"Rua Tomás IIdefonso, nº 125, quadra 05, bairro...",31/03/2021,"R$ 96.242,54",sim,"Pagamento Concluído, processo arquivado","IA = 0,20 pela OODC e\nRecuos =\n444,01m²","10,51 m (zona com gabarito 48m)",Clínica de diálise Pronefron
1,"Av. Antonio Justa, 2736, Bairro Meireles",31/03/2021,"R$ 13.405.667,44",sim,"Pagamento Concluído, processo arquivado","IA = 5,21; TO\nsubsolo =\n4,82%; AE =\n70,74m...","142,74 m (zona com gabarito 72m)",Residencial Multifamiliar - Edifício Edge
2,"Rua Doutor Gilberto Studart, nº 1589, bairro Cocó",31/03/2021,"R$2.099.464,62",sim,"Em andamento, em fase de pagamento","IA = 1,50; AE = 31,68m e\nRecuos =\n118,91m²","103,68 m (zona com gabarito 72m)",Victoria Park Residence
3,"Rua Libaneses, n° 197, lote 1, quadra 6, Bairr...",31/03/2021,"R$ 258.466,82",sim,"Pagamento Concluído,\nprocesso arquivado","AE = 17,44m","65,44 m (GABARITO 48m)",Jardino Passeio Kennedy
4,"Rua Monsenhor Bruno, nº 647, bairro Meireles",31/03/2021,"R$ 4.268.303,23",Sim,"Pagamento Concluído, processo arquivado","IA = 2,39; AE = 29,29m e\nRecuos =\n421,38 m²","101,29m (gabarito 72m)",Beach Class Meireles
5,"Rua Bento Albuquerque, n° 536, bairro Cocó",29/09/2021,"R$ 2.529.066,47",sim,"Pagamento Concluído, processo arquivado","IA = 2,95; TO\nsubsolo =\n10,00%; AE =\n47,34...","119,34m (gabarito 72m)",Edifício Flora
6,"Rua Canuto de Aguiar, n°150, bairro Meireles,",15/12/2021,"R$ 5.427.741,78",sim,Em andamento,"IA = 4,31; TO\nsolo = 1,86%; TO subsolo = 14,...","115,30m (gabarito 72m)",Nome não obtido - Construtora Colmeia
7,"Avenida Beira Mar, 2600, conjunto 01, bairro M...",15/12/2021,"R$ 11.991.136,31",não,Aguardando assinatura do termo,NaN,"71,56m (gabarito 72m)",reforma e ampliação do hotel Luzeiros - 2ªtorre
8,"Rua Nunes Valente, n° 300, bairro Meireles",27/05/2022,"R$ 8.947.750,61",sim,"EM ANDAMENTO, EM FASE DE PAGAMENTO","IA = 3,91 e AE = 56,43m","151,43m (gabarito 95m)",Mansão Diogo
9,"Rua Desembargador Leite Albuquerque, s/n, bair...",29/06/2022,"R$ 3.455.465,33",sim,"EM ANDAMENTO, EM FASE DE PAGAMENTO","IA = 1,57 e AE = 19,40m","114,40m (gabarito 95m)",BS Gold


In [169]:
ooau['geocode_result'] = ooau.apply(lambda x: gmaps.geocode(x['Endereço']), axis=1)

In [170]:
ooau['len_busca_google'] = ooau['geocode_result'].apply(lambda x: len(x))

In [171]:
ooau['len_busca_google'].value_counts()

1    23
3     2
Name: len_busca_google, dtype: int64

In [172]:
ooau.loc[ooau['len_busca_google']>1]

,Endereço,Mês/ano,Valor da OOAU,Termo de compromisso,Pagamento,O que outorgou,Altura proposta,Observações,geocode_result,len_busca_google
15,"Avenida Dioguinho, s/n, bairro Praia do Futuro II",06/09/2022,"R$ 489.367,56",sim,"CONCLUÍDO O PAGAMENTO, PROCESSO FINALIZADO","AE = 11,52m","47,52m (gabarito 36m)",NaN,[{'address_components': [{'long_name': 'Avenid...,3
21,"Avenida Dioguinho, s/n, bairro Praia do Futuro II",31/01/2023,"R$ 242.529,06",sim,VAI INICIAR O PAGAMENTO NO PRAZO DO TERMO,"AE = 8,54m","44,54m (gabarito 36m)",NaN,[{'address_components': [{'long_name': 'Avenid...,3


In [173]:
ooau = ooau.explode('geocode_result')

In [174]:
def getBairro(address_components):

    for component in address_components:
        if component['types'] == ['political', 'sublocality', 'sublocality_level_1']:
            return component['long_name']

    return 'Não identificado'

In [175]:
ooau['bairro_maps'] = ooau['geocode_result'].apply(lambda x: getBairro(x['address_components']))

In [176]:
ooau['lat_lng'] = ooau['geocode_result'].apply(lambda x: x['geometry']['location'])

In [177]:
ooau['lat'] = ooau['lat_lng'].apply(lambda x: x['lat'])
ooau['lng'] = ooau['lat_lng'].apply(lambda x: x['lng'])

In [178]:
ooau['altura_proposta'] = ooau['Altura proposta'].apply(lambda x: x.split('m')[0].replace(',','.'))

In [179]:
ooau['dtg'] = pd.to_datetime(ooau['Mês/ano'], format='%d/%m/%Y')

In [180]:
ooau['valor_ooau_tratrado'] = ooau['Valor da OOAU'].apply(lambda x: float(x.replace('R$','').replace('.','').replace(',','.').replace('(*)','')))

In [181]:
ooau['O que outorgou']

0             IA = 0,20 pela OODC e\nRecuos =\n444,01m²
1     IA =  5,21; TO\nsubsolo =\n4,82%; AE =\n70,74m...
2          IA = 1,50; AE = 31,68m e\nRecuos =\n118,91m²
3                                           AE = 17,44m
4        IA =  2,39; AE = 29,29m e\nRecuos =\n421,38 m²
5     IA =  2,95; TO\nsubsolo =\n10,00%; AE =\n47,34...
6     IA =  4,31; TO\nsolo = 1,86%; TO subsolo = 14,...
7                                                   NaN
8                               IA = 3,91 e AE = 56,43m
9                               IA = 1,57 e AE = 19,40m
10                        IA = 0,55 e FL = 139 unidades
11                                           AE = 4,44m
12                              IA = 1,66 e AE = 46,58m
13    IA= 7,10; TO solo=0,67%; TO subsolo = 9,71%; A...
14    IA= 8,41; TO solo=6,12%; TO subsolo = 11,80%; ...
15                                          AE = 11,52m
15                                          AE = 11,52m
15                                          AE =

In [182]:
ooau['O que outorgou']= ooau['O que outorgou'].fillna('=-')

In [183]:
ooau.head(1)

,Endereço,Mês/ano,Valor da OOAU,Termo de compromisso,Pagamento,O que outorgou,Altura proposta,Observações,geocode_result,len_busca_google,bairro_maps,lat_lng,lat,lng,altura_proposta,dtg,valor_ooau_tratrado
0,"Rua Tomás IIdefonso, nº 125, quadra 05, bairro...",31/03/2021,"R$ 96.242,54",sim,"Pagamento Concluído, processo arquivado","IA = 0,20 pela OODC e\nRecuos =\n444,01m²","10,51 m (zona com gabarito 48m)",Clínica de diálise Pronefron,{'address_components': [{'long_name': 'quadra ...,1,Cambeba,"{'lat': -3.8117034, 'lng': -38.483229}",-3.81,-38.48,10.51,2021-03-31,96242.54


In [184]:
ooau['Descrição'] = ooau.apply(lambda x: f'<b>{x["Observações"]}</b><br><b>Valor da OOAU:</b>{x["Valor da OOAU"]}<br><b>Altura proposta:</b>{x["Altura proposta"]}<br><b>Endereço:</b>{x["Endereço"]}<br><b>O que outorgou</b>{x["O que outorgou"]}<br>', axis=1)

In [185]:
# ooau['Descrição'] = ooau['Observações'] + '\nO que outorgou:' + ooau['O que outorgou']+'Pagamento: '+ooau['Pagamento']

In [186]:
ooau.head(5)

,Endereço,Mês/ano,Valor da OOAU,Termo de compromisso,Pagamento,O que outorgou,Altura proposta,Observações,geocode_result,len_busca_google,bairro_maps,lat_lng,lat,lng,altura_proposta,dtg,valor_ooau_tratrado,Descrição
0,"Rua Tomás IIdefonso, nº 125, quadra 05, bairro...",31/03/2021,"R$ 96.242,54",sim,"Pagamento Concluído, processo arquivado","IA = 0,20 pela OODC e\nRecuos =\n444,01m²","10,51 m (zona com gabarito 48m)",Clínica de diálise Pronefron,{'address_components': [{'long_name': 'quadra ...,1,Cambeba,"{'lat': -3.8117034, 'lng': -38.483229}",-3.81,-38.48,10.51,2021-03-31,96242.54,<b>Clínica de diálise Pronefron</b><br><b>Valo...
1,"Av. Antonio Justa, 2736, Bairro Meireles",31/03/2021,"R$ 13.405.667,44",sim,"Pagamento Concluído, processo arquivado","IA = 5,21; TO\nsubsolo =\n4,82%; AE =\n70,74m...","142,74 m (zona com gabarito 72m)",Residencial Multifamiliar - Edifício Edge,"{'address_components': [{'long_name': '2736', ...",1,Meireles,"{'lat': -3.7275791, 'lng': -38.49518880000001}",-3.73,-38.50,142.74,2021-03-31,13405667.44,<b>Residencial Multifamiliar - Edifício Edge</...
2,"Rua Doutor Gilberto Studart, nº 1589, bairro Cocó",31/03/2021,"R$2.099.464,62",sim,"Em andamento, em fase de pagamento","IA = 1,50; AE = 31,68m e\nRecuos =\n118,91m²","103,68 m (zona com gabarito 72m)",Victoria Park Residence,"{'address_components': [{'long_name': '1589', ...",1,Cocó,"{'lat': -3.7455093, 'lng': -38.474417}",-3.75,-38.47,103.68,2021-03-31,2099464.62,<b>Victoria Park Residence</b><br><b>Valor da ...
3,"Rua Libaneses, n° 197, lote 1, quadra 6, Bairr...",31/03/2021,"R$ 258.466,82",sim,"Pagamento Concluído,\nprocesso arquivado","AE = 17,44m","65,44 m (GABARITO 48m)",Jardino Passeio Kennedy,{'address_components': [{'long_name': 'Preside...,1,Presidente Kennedy,"{'lat': -3.7278919, 'lng': -38.5692627}",-3.73,-38.57,65.44,2021-03-31,258466.82,<b>Jardino Passeio Kennedy</b><br><b>Valor da ...
4,"Rua Monsenhor Bruno, nº 647, bairro Meireles",31/03/2021,"R$ 4.268.303,23",Sim,"Pagamento Concluído, processo arquivado","IA = 2,39; AE = 29,29m e\nRecuos =\n421,38 m²","101,29m (gabarito 72m)",Beach Class Meireles,"{'address_components': [{'long_name': '647', '...",1,Meireles,"{'lat': -3.7281884, 'lng': -38.50789109999999}",-3.73,-38.51,101.29,2021-03-31,4268303.23,<b>Beach Class Meireles</b><br><b>Valor da OOA...


In [187]:
df_ooau = ooau[['dtg','bairro_maps','valor_ooau_tratrado','lat','lng','Descrição']]

In [188]:
df_ooau['tipo_outorga'] = 'OOAU'

<ipython-input-188-8c39f6758c16>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ooau['tipo_outorga'] = 'OOAU'


In [189]:
df_ooau.groupby(['bairro_maps'])['valor_ooau_tratrado'].sum().reset_index().sort_values(['valor_ooau_tratrado'],ascending=False)

,bairro_maps,valor_ooau_tratrado
7,Meireles,129202885.31
8,Mucuripe,24241872.48
9,Praia de Iracema,23502813.01
5,Cocó,9674813.77
4,Centro,8787085.77
0,Aldeota,3455465.33
1,Antonio Diogo,2195689.86
2,Barroso,805244.86
11,São João do Tauape,710950.44
6,Itaperi,425727.31


### Base final

In [190]:
df_ooau.head(2)

,dtg,bairro_maps,valor_ooau_tratrado,lat,lng,Descrição,tipo_outorga
0,2021-03-31,Cambeba,96242.54,-3.81,-38.48,<b>Clínica de diálise Pronefron</b><br><b>Valo...,OOAU
1,2021-03-31,Meireles,13405667.44,-3.73,-38.50,<b>Residencial Multifamiliar - Edifício Edge</...,OOAU


In [191]:
df_oodc = oodc[['Nome','Bairro','Descrição','lat','lng','ATIVIDADE','valor_tratado','ano_ref ']]

In [192]:
df_oodc['dtg'] = pd.to_datetime(df_oodc['ano_ref '], format='%Y')
df_oodc['tipo_outorga'] = 'OODC'

<ipython-input-192-1a0c4d762931>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oodc['dtg'] = pd.to_datetime(df_oodc['ano_ref '], format='%Y')
<ipython-input-192-1a0c4d762931>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oodc['tipo_outorga'] = 'OODC'


In [193]:
df_oodc = df_oodc[['dtg','Bairro','valor_tratado','lat', 'lng', 'Descrição','tipo_outorga']]

In [194]:
df_oodc.columns=['dtg','Bairro','valor_tratado','lat', 'lng', 'Descrição','tipo_outorga']
df_ooau.columns=['dtg','Bairro','valor_tratado','lat', 'lng', 'Descrição','tipo_outorga']

In [195]:
df_outorga_onerosa = pd.concat([df_oodc, df_ooau])

In [196]:
df_outorga_onerosa.to_csv(path2+'df_outorga_onerosa.csv')

In [197]:
df_outorga_onerosa.to_csv('df_outorga_onerosa.csv')

In [198]:
df_outorga_onerosa.groupby('Bairro')['valor_tratado'].sum().reset_index().sort_values(by='valor_tratado',ascending=False)

,Bairro,valor_tratado
18,Meireles,134134437.88
19,Mucuripe,24609248.60
22,Praia de Iracema,23502813.01
0,Aldeota,14169329.30
7,Cocó,12632253.70
5,Centro,8829067.71
1,Antonio Diogo,2195689.86
26,São João do Tauape,1573578.12
11,Guararapes,1536067.39
8,Dionísio Torres,1240069.42


In [199]:
df_outorga_onerosa.groupby('Bairro')['valor_tratado'].sum().reset_index().sort_values(by='valor_tratado',ascending=False).to_csv('resumo_total_bairro.csv')

In [200]:
resumo_bairros = df_outorga_onerosa.groupby(['Bairro','tipo_outorga'])['valor_tratado'].sum().reset_index().pivot(index='Bairro',columns='tipo_outorga').fillna(0)
resumo_bairros = resumo_bairros.reset_index()
resumo_bairros.columns = ['Bairro','Valor OOAU','Valor OODC']

In [201]:
resumo_bairros['Total'] = resumo_bairros['Valor OOAU'] +  resumo_bairros['Valor OODC']

In [202]:
resumo_bairros.sort_values(by='Total',ascending=False)

,Bairro,Valor OOAU,Valor OODC,Total
18,Meireles,129202885.31,4931552.57,134134437.88
19,Mucuripe,24241872.48,367376.12,24609248.60
22,Praia de Iracema,23502813.01,0.00,23502813.01
0,Aldeota,3455465.33,10713863.97,14169329.30
7,Cocó,9674813.77,2957439.93,12632253.70
5,Centro,8787085.77,41981.94,8829067.71
1,Antonio Diogo,2195689.86,0.00,2195689.86
26,São João do Tauape,710950.44,862627.68,1573578.12
11,Guararapes,0.00,1536067.39,1536067.39
8,Dionísio Torres,0.00,1240069.42,1240069.42


In [203]:
resumo_bairros.sort_values(by='Total',ascending=False).to_csv(path2+'resumo_outorga_bairro.csv')

In [204]:
resumo_bairros.sort_values(by='Total',ascending=False).to_csv('resumo_outorga_bairro.csv')